# 找出所有集章點（開放未開放都有）

In [128]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# 基本設定
base_url = "https://platinumaps.jp/d/seventeen-thecity2024?culture=ja&spot="
starting_spot = 400000
basic_title = "SEVENTEEN [RIGHT HERE] THE CITY RALLY | プラチナマップ"

# 用於儲存結果的列表
results = []

# Headers 設定
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# 爬取函數
def fetch_spot_data(spot_number):
    url = base_url + str(spot_number)
    try:
        response = requests.get(url, headers=headers, timeout=5)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        # 提取標題
        full_title = soup.title.string if soup.title else ""
        title = full_title.replace(" - SEVENTEEN [RIGHT HERE] THE CITY RALLY | プラチナマップ", "")

        # 判斷標題是否符合基本標題
        if full_title != basic_title:
            print(f"Spot name: {spot_number} is {title}")
            return {"Spot": spot_number, "Title": title}
        else:
            return None
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch spot {spot_number}: {e}")
        return None

# 使用多线程加速
print("Start fetching website & place names")

with ThreadPoolExecutor(max_workers=100) as executor:  # 控制並發數量
    futures = [executor.submit(fetch_spot_data, starting_spot + n) for n in range(10000)]
    for future in futures:
        result = future.result()
        if result:
            results.append(result)

# 將結果存入 DataFrame
df = pd.DataFrame(results)

# 保存為 CSV 檔案
csv_file = "spots_results.csv"
df.to_csv(csv_file, index=False, encoding="utf-8-sig")
print(f"Results saved to {csv_file}")


Start fetching website & place names
Spot name: 400609 is メ～テレ 本社南側
Spot name: 400615 is 中日ビル 7階屋上広場Spot name: 400610 is ラシック 5F 久屋大通側吹き抜けSpot name: 400621 is セントラルパーク ロッテリア前円柱ポスターパネル
Spot name: 400611 is 名古屋栄三越 3F カスタマーステーション横


Spot name: 400618 is 中部電力 MIRAI TOWER 1階エントランス
Spot name: 400628 is Hisaya Central Vision
Spot name: 400624 is 豊田合成リンク受付テント付近
Spot name: 400809 is 大名古屋ビル Shops & Restaurants
Spot name: 401340 is 阪急三番街 北館B1Fニトリ デコホーム前Spot name: 401343 is ディアモール大阪 ファッショナブルストリートの中央
Spot name: 401347 is NU茶屋町 6F 『TOWER RECORDS』店内

Spot name: 401346 is グランフロント大阪北館5F HMV
Spot name: 401345 is グランフロント大阪北館6F ウメキタフロア ギャラリースペース
Spot name: 401348 is NU茶屋町プラス2F 『WOCCA ROCCA』前
Spot name: 402697 is 矢場とん【共通スタンプ※対象店舗7店】
Spot name: 402720 is NTTドコモ：ドコモショップ／d garden【共通スタンプ※対象店舗3店】
Spot name: 403127 is Sunshine City 専門店街アルパ3F 広小路(北側)
Spot name: 403315 is バンテリンドーム ナゴヤ
Spot name: 403316 is みずほPayPayドーム福岡Spot name: 403313 is 京セラドーム大阪

Spot name: 403314 is 東京ドーム
Spot name: 403306 is かくれボンボンイ(イオンモール N

# 
# 把所有「目前」開放的集章點的「地址」抓下來

In [131]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from concurrent.futures import ProcessPoolExecutor
import multiprocessing


def init_driver():
    """初始化 Selenium WebDriver"""
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--disk-cache-dir=/path/to/cache")
    options.add_argument("--enable-application-cache")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.page_load_strategy = 'eager'

    prefs = {
        "profile.managed_default_content_settings.images": 2,  # 禁用圖片
    }
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(executable_path="./chromedriver", options=options)
    return driver


def fetch_address(row):
    """爬取地址信息"""
    spot = row['Spot']
    title = row['Title']
    base_url = "https://platinumaps.jp/d/seventeen-thecity2024?culture=ja&spot="
    url = base_url + str(spot)
    address = ""

    try:
        driver = init_driver()  # 每个进程初始化自己的 WebDriver
        driver.get(url)
        print(f"Accessing {url}")

        # 等待 iframe 出現並切換
        iframe_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
        driver.switch_to.frame(iframe_element)

        # 檢查指定的元素是否存在
        try:
            element_exists = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "body > div > div > div.platinumap > div.poiselector"))
            )
            if element_exists:
                print("Main element exists, proceeding to find address...")
                
                # 嘗試查找第一個地址
                try:
                    address_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "body > div > div > div.platinumap > div.poiselector > div > div > div > div > div.detail__inner > div.scrollview__wrapper.detail__scrollview.-v > div > div > div > div.poiproperties > table > tbody > tr:nth-child(1) > td > div > address"))
                    )
                    address = address_element.text.strip()
                    print(f"Address found using first selector: {address}")
                except Exception:
                    print("First address not found, trying second selector...")

                    # 嘗試第二個地址
                    try:
                        address_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, "body > div > div > div.platinumap > div.poiselector > div > div > div > div > div.detail__inner > div.scrollview__wrapper.detail__scrollview.-v > div > div > div > div.poiproperties > table > tbody > tr > td > div > address"))
                        )
                        address = address_element.text.strip()
                        print(f"Address found using second selector: {address}")
                    except Exception:
                        print("Address not found using second selector either.")
            else:
                print("Main element does not exist, skipping address search.")
        except Exception:
            print("Main element does not exist, skipping address search.")

    except Exception as e:
        print(f"Failed to fetch data for Spot {spot}: {e}")
    finally:
        driver.quit()  # 確保關閉 WebDriver

    return {"Spot": spot, "Title": title, "Address": address}


def getAddress():
    # 讀取保存的表格
    csv_file = "spots_results.csv"
    df = pd.read_csv(csv_file)

    # 使用多進程加速爬取
    print("Starting parallel scraping...")
    with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
        results = list(executor.map(fetch_address, df.to_dict(orient="records")))

    # 保存結果
    updated_df = pd.DataFrame(results)
    output_csv_file = "updated_spots_results.csv"
    updated_df.to_csv(output_csv_file, index=False, encoding="utf-8-sig")
    print(f"Updated results saved to {output_csv_file}")


In [132]:
getAddress()

Starting parallel scraping...
Accessing https://platinumaps.jp/d/seventeen-thecity2024?culture=ja&spot=400624Accessing https://platinumaps.jp/d/seventeen-thecity2024?culture=ja&spot=400809

Accessing https://platinumaps.jp/d/seventeen-thecity2024?culture=ja&spot=400618
Accessing https://platinumaps.jp/d/seventeen-thecity2024?culture=ja&spot=401340Accessing https://platinumaps.jp/d/seventeen-thecity2024?culture=ja&spot=400610

Accessing https://platinumaps.jp/d/seventeen-thecity2024?culture=ja&spot=401346Accessing https://platinumaps.jp/d/seventeen-thecity2024?culture=ja&spot=400615

Accessing https://platinumaps.jp/d/seventeen-thecity2024?culture=ja&spot=400609
Accessing https://platinumaps.jp/d/seventeen-thecity2024?culture=ja&spot=401343
Accessing https://platinumaps.jp/d/seventeen-thecity2024?culture=ja&spot=401347
Accessing https://platinumaps.jp/d/seventeen-thecity2024?culture=ja&spot=400628
Accessing https://platinumaps.jp/d/seventeen-thecity2024?culture=ja&spot=401345
Accessing 

# 
# 地址轉換成經緯度

In [133]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time

# 读取包含地址的 CSV 文件
input_csv = 'updated_spots_results.csv'  # 替换为您的输入文件名
df = pd.read_csv(input_csv)

# 初始化 Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(executable_path="./chromedriver", options=options)

# 打开 MapLocation 网站
driver.get("https://maplocation.sjfkai.com")

time.sleep(2)

# 用于存储结果的列表
results = []

for index, row in df.iterrows():
    address = row['Address'] 
    spot = row['Spot']
    title = row['Title']

    try:
        # 输入地址
        textarea = driver.find_element(By.TAG_NAME, "textarea")
        textarea.send_keys(address)        
        textarea.send_keys(Keys.ENTER)

    except Exception as e:
        textarea.send_keys(" ")        
        textarea.send_keys(Keys.ENTER)

try:
    # 选择平台（例如，选择 Google）
    platform_select = driver.find_element(
        By.CSS_SELECTOR, "#platform > div:nth-child(2) > label > span.ant-radio > input")
    platform_select.click()

    # 点击转换按钮
    convert_button = driver.find_element(
        By.CSS_SELECTOR, "#root > div > section > main > form > div > div.ant-col.ant-col-xs-24.ant-col-md-8 > div > div:nth-child(3) > div > div > div > span > button")
    convert_button.click()

except Exception as e:
        print(f"Error press key")

# 等待结果加载
time.sleep(80)  # 根据网络情况调整等待时间

for index, row in df.iterrows():
    address = row['Address'] 
    spot = row['Spot']
    title = row['Title']

    results.append({
        "Spot": spot,
        "Title": title,
        "Address": address,
        "Longitude": " ",
        "Latitude": " ",
        "Available": "not yet"

    })
    try:
        # 等待表格加载
        result_table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//table/tbody"))
        )

        # 抓取所有行
        rows = result_table.find_elements(By.TAG_NAME, "tr")
        print(f"Found {len(rows)} rows in the table.")  # 调试信息

        # 遍历表格行
        match_found = False
        for row in rows:
            cols = row.find_elements(By.TAG_NAME, "td")
            result_address = cols[1].text.strip()
            longitude = cols[2].text.strip()
            latitude = cols[3].text.strip()

            # 匹配输入地址
            if result_address == address:
                results[index]['Longitude'] = longitude
                results[index]['Latitude'] = latitude
                results[index]['Available'] = "start"


                print(f"Match found for Spot {spot}: {longitude}, {latitude}")
                match_found = True
                break

    except Exception as e:
        print(f"Error processing address '{address}': {e}")
        results.append({
            "Spot": spot,
            "Title": title,
            "Address": address,
            "Longitude": "",
            "Latitude": "",
            "Available": "not yet"

        })


# 关闭浏览器
driver.quit()

# 将结果保存到新的 CSV 文件
output_csv = 'updated_spots_results_with_coordinates.csv'  # 替换为您的输出文件名
results_df = pd.DataFrame(results)
results_df.to_csv(output_csv, index=False, encoding='utf-8-sig')
print(f"Results saved to {output_csv}")


Found 28 rows in the table.
Match found for Spot 400609: 136.9033681, 35.1506907
Found 28 rows in the table.
Match found for Spot 400610: 136.893677, 35.167242
Found 28 rows in the table.
Match found for Spot 400611: 136.893677, 35.167242
Found 28 rows in the table.
Match found for Spot 400615: 136.8952018, 35.1678787
Found 28 rows in the table.
Match found for Spot 400618: 136.9071723, 35.1725257
Found 28 rows in the table.
Match found for Spot 400621: 136.9076788, 35.1714486
Found 28 rows in the table.
Found 28 rows in the table.
Found 28 rows in the table.
Match found for Spot 400809: 136.8844986, 35.1720523
Found 28 rows in the table.
Found 28 rows in the table.
Found 28 rows in the table.
Found 28 rows in the table.
Found 28 rows in the table.
Found 28 rows in the table.
Found 28 rows in the table.
Found 28 rows in the table.
Match found for Spot 402720: 136.9068139, 35.1688694
Found 28 rows in the table.
Match found for Spot 403127: 139.7195415, 35.7289709
Found 28 rows in the ta

# 
# 輸出成google map可閱讀的格式

## 存「已出現」地點與經緯度（可忽視）

In [134]:
input_csv = 'updated_spots_results_with_coordinates.csv'  # 替换为您的输入文件名
df = pd.read_csv(input_csv)


results = []

for index, row in df.iterrows():
    longitude = row['Longitude'] 

    if longitude != " ":
        results.append({
            "Spot": row['Spot'],
            "Title": row['Title'],
            "Address":row['Address'],
            "Longitude": row['Longitude'],
            "Latitude": row['Latitude']

        })
# 将结果保存到新的 CSV 文件
output_csv = 'google_maps_place_NOW.csv'  # 替换为您的输出文件名
results_df = pd.DataFrame(results)
results_df.to_csv(output_csv, index=False, encoding='utf-8-sig')
print(f"Results saved to {output_csv}")


Results saved to google_maps_place_NOW.csv


## 存「已出現」地點的 QRCODE（圖檔有經緯度 / 標題描述）

In [135]:
import qrcode
from PIL import Image
import piexif
import pandas as pd
import os
import shutil

def clear_qrcode_folder(folder_path):
    """
    清空指定的文件夹
    :param folder_path: 文件夹路径
    """
    if os.path.exists(folder_path):
        # 删除文件夹中的所有内容
        shutil.rmtree(folder_path)
        print(f"Cleared folder: {folder_path}")
    # 重新创建空文件夹
    os.makedirs(folder_path, exist_ok=True)
    print(f"Recreated folder: {folder_path}")

# 指定二维码存放的文件夹
qrcode_folder = "./qrcode"
clear_qrcode_folder(qrcode_folder)


def decimal_to_dms(decimal):
    """
    将十进制经纬度转换为度、分、秒格式，适用于 piexif.
    :param decimal: float, 十进制经纬度
    :return: tuple, (degrees, minutes, seconds) 格式的分数表示
    """
    decimal = abs(decimal)
    degrees = int(decimal)
    minutes = int((decimal - degrees) * 60)
    seconds = round((decimal - degrees - minutes / 60) * 3600 * 10000)  # 精度放大到万分位
    return (
        (degrees, 1),
        (minutes, 1),
        (seconds, 10000)
    )


# 输入网址
base_url = "https://platinumaps.jp/d/seventeen-thecity2024?culture=ja&spot="
key_url = "&sr=1047&srcheckin=1"

# 创建 KML 对象
kml = simplekml.Kml()

input_csv = 'updated_spots_results_with_coordinates.csv'  # 替换为您的输入文件名
df = pd.read_csv(input_csv)

for index, row in df.iterrows():
    available = "no"
    spot = row['Spot']
    title = row['Title']  # 获取 Title
    address = row['Address']
    url = base_url + str(spot) + key_url

    exif_dict = {"0th": {}, "Exif": {}, "GPS": {}, "1st": {}, "thumbnail": None} 
    
    if row['Available'] == "start":
        latitude = float(row['Latitude'])
        longitude = float(row['Longitude'])
        latitude_dms = decimal_to_dms(latitude) 
        longitude_dms = decimal_to_dms(longitude)
        
            # 设置 GPS 信息
        gps_info = { 
            piexif.GPSIFD.GPSLatitude: latitude_dms, 
            piexif.GPSIFD.GPSLongitude: longitude_dms, 
            piexif.GPSIFD.GPSAltitude: (500, 1), 
            piexif.GPSIFD.GPSLatitudeRef: 'N' if latitude >= 0 else 'S', 
            piexif.GPSIFD.GPSLongitudeRef: 'E' if longitude >= 0 else 'W',
        }
        exif_dict['GPS'] = gps_info 

        # 处理 Title，确保其编码符合 EXIF 要求
        exif_dict["0th"][piexif.ImageIFD.ImageDescription] = str(spot)

        # 在 EXIF 中额外存储经纬度的小数值
        user_comment = f"Latitude: {latitude}, Longitude: {longitude}".encode("utf-8")
        exif_dict["Exif"][piexif.ExifIFD.UserComment] = user_comment

        # 生成 EXIF 字节
        exif_bytes = piexif.dump(exif_dict)

        # 创建二维码
        qr = qrcode.QRCode(
            version=1,  # 控制二维码的大小，1 是最小的，值越大二维码越复杂
            error_correction=qrcode.constants.ERROR_CORRECT_L,  # 纠错级别 (L, M, Q, H)
            box_size=10,  # 每个点的像素大小
            border=4,  # 边框的宽度，单位为点
        )
        qr.add_data(url)  # 添加网址到二维码
        qr.make(fit=True)  # 适应大小

        # 生成二维码图像
        img = qr.make_image(fill_color="black", back_color="white")  # 设置前景色和背景色

        # 保存二维码图像并附加 EXIF 数据
        output_path = f"./qrcode/{row['Spot']}-{row['Available']}-{row['Title']}.jpg"
        img.save(output_path, exif=exif_bytes)  # 保存为图片文件
        print(f"QR code saved as {output_path}")

    else:
        latitude = 0
        longitude = 0
        latitude_dms = decimal_to_dms(latitude)
        longitude_dms = decimal_to_dms(longitude)
    
    

Cleared folder: ./qrcode
Recreated folder: ./qrcode
QR code saved as ./qrcode/400609-start-メ～テレ 本社南側.jpg
QR code saved as ./qrcode/400610-start-ラシック 5F 久屋大通側吹き抜け.jpg
QR code saved as ./qrcode/400611-start-名古屋栄三越 3F カスタマーステーション横.jpg
QR code saved as ./qrcode/400615-start-中日ビル 7階屋上広場.jpg
QR code saved as ./qrcode/400618-start-中部電力 MIRAI TOWER 1階エントランス.jpg
QR code saved as ./qrcode/400621-start-セントラルパーク ロッテリア前円柱ポスターパネル.jpg
QR code saved as ./qrcode/400809-start-大名古屋ビル Shops & Restaurants.jpg
QR code saved as ./qrcode/402720-start-NTTドコモ：ドコモショップ／d garden【共通スタンプ※対象店舗3店】.jpg
QR code saved as ./qrcode/403127-start-Sunshine City 専門店街アルパ3F 広小路(北側).jpg
QR code saved as ./qrcode/403306-start-かくれボンボンイ(イオンモール Nagoya Noritake Garden内).jpg
QR code saved as ./qrcode/403845-start-鳥貴族 名駅３丁目店.jpg
QR code saved as ./qrcode/403849-start-鳥貴族 名駅広小路店.jpg
QR code saved as ./qrcode/403851-start-鳥貴族 名駅西口店.jpg
QR code saved as ./qrcode/403853-start-鳥貴族 名駅東口店.jpg
QR code saved as ./qrcode/403855-start-鳥貴族 大曾根店.jpg

## 把QRCODE和地點資訊存成kmz格式，方便google 我的地圖導入

In [136]:
import os
import pandas as pd
import simplekml
from zipfile import ZipFile

# 输入数据的 CSV 文件
input_csv = 'updated_spots_results_with_coordinates.csv'  # 替换为您的输入文件名
df = pd.read_csv(input_csv)

# 二维码图片文件夹
qrcode_folder = "./qrcode"  # 替换为二维码图片的实际路径

# 创建 KML 对象
kml = simplekml.Kml()

# 遍历每一行，生成 KML 标记点
for index, row in df.iterrows():
    title = row['Title']
    latitude = row['Latitude']
    longitude = row['Longitude']
    spot = row['Spot']
    address = row['Address']
    available = row['Available']

    # 二维码文件名
    qrcode_file = f"{spot}-{available}-{title}.jpg"

    # 跳过无效数据
    if pd.isna(latitude) or pd.isna(longitude) or latitude == " " or longitude == " ":
        continue

    # 创建标记点
    pnt = kml.newpoint(name=title)
    pnt.coords = [(float(longitude), float(latitude))]  # 经纬度坐标
    qrcode_relative_path = f"files/{qrcode_file}"  # 相对于 KMZ 的路径

    pnt.description = (
        f"<b>Title:</b> {title}<br>"
        f"<b>Lat:</b> {latitude}, <b>Lon:</b> {longitude}<br>"
        f"<b>SpotCode:</b> {spot}<br>"
        f"<b>Address:</b> {address}<br>"
        f"<b>QR Code:</b><br>"
        f"<img src='{qrcode_relative_path}' width='256' height='256'>"
    )  # 描述信息，嵌入二维码图片

    # 添加二维码图片为图标
    qrcode_path = os.path.join(qrcode_folder, qrcode_file)
    if os.path.exists(qrcode_path):
        pnt.style.iconstyle.icon.href = qrcode_relative_path
    else:
        print(f"QR code image not found for {title}: {qrcode_file}")

# 保存为 KML 文件
kml_file = "map_with_qrcodes.kml"
kml.save(kml_file)
print(f"KML file saved as {kml_file}")

# 将 KML 和二维码图片打包成 KMZ 文件
kmz_file = "map_with_qrcodes.kmz"
with ZipFile(kmz_file, 'w') as kmz:
    # 添加 KML 文件
    kmz.write(kml_file, os.path.basename(kml_file))

    # 添加所有二维码图片
    for file in os.listdir(qrcode_folder):
        if file.endswith(".jpg"):  # 确保只添加 JPG 文件
            kmz.write(os.path.join(qrcode_folder, file), f"files/{file}")

print(f"KMZ file saved as {kmz_file}")


KML file saved as map_with_qrcodes.kml
KMZ file saved as map_with_qrcodes.kmz


## 純「已出現」地點的 GPX描述檔生成（直接導入xcode）

In [2]:
import pandas as pd
import os

def clear_folder(folder_path):
    """
    清空指定的文件夹
    :param folder_path: 文件夹路径
    """
    if os.path.exists(folder_path):
        # 删除文件夹中的所有内容
        shutil.rmtree(folder_path)
        print(f"Cleared folder: {folder_path}")
    # 重新创建空文件夹
    os.makedirs(folder_path, exist_ok=True)
    print(f"Recreated folder: {folder_path}")

# 读取包含经纬度的 CSV 文件
input_csv = 'updated_spots_results_with_coordinates.csv'  # 替换为您的输入文件名
df = pd.read_csv(input_csv)

# 创建 GPX 文件的函数
def create_gpx_file(title, latitude, longitude, output_folder="./gpsPosition"):
    """
    创建一个 GPX 文件
    :param title: 文件名
    :param latitude: 纬度
    :param longitude: 经度
    :param output_folder: 存储 GPX 文件的文件夹
    """
    # 如果经纬度无效，跳过创建
    if pd.isna(latitude) or pd.isna(longitude) or latitude == " " or longitude == " ":
        print(f"Skipping invalid coordinates for {title}")
        return

    # 创建 GPX 文件内容
    gpx_content = f"""<?xml version="1.0"?>
<gpx version="1.1" creator="gpxgenerator.com">
  <wpt lat="{latitude}" lon="{longitude}">
    <name>{title}</name>
  </wpt>
</gpx>"""

    # 生成文件路径
    output_file = f"{output_folder}/{title}.gpx"

    # 保存文件
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(gpx_content)
    print(f"GPX file saved: {output_file}")


    
folder = "./svt 集章 gps/svt 集章 gps/gpsPosition"
clear_folder(folder)

# 遍历 CSV 文件，生成每个点的 GPX 文件
for index, row in df.iterrows():
    title = row['Title']  # 从 CSV 获取标题
    latitude = row['Latitude']  # 从 CSV 获取纬度
    longitude = row['Longitude']  # 从 CSV 获取经度

    # 创建 GPX 文件
    create_gpx_file(title, latitude, longitude)


Recreated folder: ./svt 集章 gps/svt 集章 gps/gpsPosition
GPX file saved: ./gpsPosition/メ～テレ 本社南側.gpx
GPX file saved: ./gpsPosition/ラシック 5F 久屋大通側吹き抜け.gpx
GPX file saved: ./gpsPosition/名古屋栄三越 3F カスタマーステーション横.gpx
GPX file saved: ./gpsPosition/中日ビル 7階屋上広場.gpx
GPX file saved: ./gpsPosition/中部電力 MIRAI TOWER 1階エントランス.gpx
GPX file saved: ./gpsPosition/セントラルパーク ロッテリア前円柱ポスターパネル.gpx
Skipping invalid coordinates for 豊田合成リンク受付テント付近
Skipping invalid coordinates for Hisaya Central Vision
GPX file saved: ./gpsPosition/大名古屋ビル Shops & Restaurants.gpx
Skipping invalid coordinates for 阪急三番街 北館B1Fニトリ デコホーム前
Skipping invalid coordinates for ディアモール大阪 ファッショナブルストリートの中央
Skipping invalid coordinates for グランフロント大阪北館6F ウメキタフロア ギャラリースペース
Skipping invalid coordinates for グランフロント大阪北館5F HMV
Skipping invalid coordinates for NU茶屋町 6F 『TOWER RECORDS』店内
Skipping invalid coordinates for NU茶屋町プラス2F 『WOCCA ROCCA』前
Skipping invalid coordinates for 矢場とん【共通スタンプ※対象店舗7店】
GPX file saved: ./gpsPosition/NTTドコモ：ドコモショップ／d garden【共通スタンプ※対